In [3]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Load datasets
ratings = pd.read_csv('/content/ratings.csv')  # userId, movieId, rating
movies = pd.read_csv('/content/movies.csv')    # movieId, title

# Create user-item rating matrix
user_movie_matrix = ratings.pivot_table(index='userId', columns='movieId', values='rating')
user_movie_matrix_filled = user_movie_matrix.fillna(0)

# Compute user similarity
user_similarity = cosine_similarity(user_movie_matrix_filled)
user_similarity_df = pd.DataFrame(user_similarity, index=user_movie_matrix.index, columns=user_movie_matrix.index)

# Function to get similar users
def get_similar_users(user_id, top_n=5):
    if user_id not in user_similarity_df.index:
        return []
    sim_users = user_similarity_df[user_id].sort_values(ascending=False)
    return sim_users.iloc[1:top_n+1].index  # exclude self

# Function to recommend movies
def recommend_movies(user_id, num_recommendations=5):
    if user_id not in user_movie_matrix.index:
        return "User ID not found. Please try a different one."

    similar_users = get_similar_users(user_id)
    if not similar_users.any():
        return "No similar users found for recommendations."

    user_ratings = user_movie_matrix.loc[similar_users].mean().sort_values(ascending=False)

    already_rated = user_movie_matrix.loc[user_id].dropna().index
    recommendations = user_ratings.drop(index=already_rated, errors='ignore').head(num_recommendations)

    return movies[movies['movieId'].isin(recommendations.index)][['title']]

# --- MAIN EXECUTION BLOCK ---

try:
    user_input = int(input("Enter your User ID: "))
    print("\nRecommended Movies for You:")
    print(recommend_movies(user_input))
except ValueError:
    print("Invalid input. Please enter a numeric user ID.")


Enter your User ID: 77

Recommended Movies for You:
                                                 title
257                                Pulp Fiction (1994)
7845                             21 Jump Street (2012)
7927                    Amazing Spider-Man, The (2012)
7988           Perks of Being a Wallflower, The (2012)
9618  Three Billboards Outside Ebbing, Missouri (2017)
